# Create a Dashboard Using Coronavirus Dataset

This project shows how to make a dashboard in Python using coronavirus dataset obtained from [EU Open Data Portal - COVID-19 Coronavirus Data ](https://data.europa.eu/euodp/en/data/dataset/covid-19-coronavirus-data).

Dash apps are composed of 2 parts: layout and interactivity. 
Layout can be set by html components.
Interactivity can be achieved using callbacks.


**Sources:**

1. https://dash.plotly.com/dash-core-components/tabs
2. https://medium.com/swlh/dashboards-in-python-for-beginners-and-everyone-else-using-dash-f0a045a86644
3. https://medium.com/a-r-g-o/using-plotlys-dash-to-deliver-public-sector-decision-support-dashboards-ac863fa829fb
4. https://www.w3schools.com/bootstrap/bootstrap_grid_system.asp

### 1. Import Necessary Libraries

In [2]:
import pandas as pd
import dash
import jupyter_dash

#These libraries are for the layout (visual components of the application) :
import dash_core_components as dcc #describe higher-level components that are interactive and are generated with 
                                   #JavaScript, HTML, and CSS through the React.js library.
import dash_html_components as html
from dash.dependencies import Input, Output # needed for callbacks
import dash_table # needed to make a table in dashboard
import plotly.express as px # needed to make map
import plotly.io as plt_io

### 2. Load the Data

In [28]:
# load data
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

### 3. Clean Data

In [29]:
# clean data
df = df.rename(columns={'dateRep':'Date','countriesAndTerritories':'Country',
                        'countryterritoryCode':'Code','popData2018':'Population',
                        'cases':'Cases', 'deaths':'Deaths'})
# change date time
df.Date = pd.to_datetime(df.Date, format='%d/%m/%Y')

In [30]:
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.loc[df['Cases']<0, 'Cases'] = 0
df.loc[df['Deaths']<0, 'Deaths'] = 0

In [31]:
# make a list of all countries
country = df['Country'].unique()

In [32]:
# create a table for Data Table tab in dashboard
df_tab = df.groupby('Country')[['Cases','Deaths']].sum().reset_index()
df_tab.sort_values(by='Cases', ascending=False, inplace=True)

In [33]:
# create table for map
df_map = df.groupby('Country').agg({'Cases':'sum', 'Code':'max'}).reset_index()

In [35]:
today = df['Date'].max().strftime('%d %b %Y')
today

'15 Jul 2020'

### 3. Build Dashboard

In [37]:
# build app viewer
#viewer = AppViewer() 

# build app
# set the fonts in app with external stylesheets(Bootstrap Grid System is used)
#external_stylesheets = ['https://raw.githubusercontent.com/plotly/dash-oil-and-gas-demo/master/assets/s1.css']
app = jupyter_dash.JupyterDash(__name__)

colors = {
    'background': '#ffffff',
    'text': '#070707'
}


# create a buble map showing cases by country
fig = px.scatter_geo(df_map, locations="Code",
                     hover_name="Country", size="Cases",
                     projection="natural earth",
                     title = 'Covid-19 Cases in the World')
fig.update_geos(showcountries=True) # show country borders
fig.update_layout(title=dict(x=0.5), plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])




app.layout = html.Div(
    [

        html.Div(
            [
               html.H3(
                   'Coronavirus Statistics',
                   style={
                       'textAlign': 'center',
                       'color': colors['text']
                       }
                   )
            ],
            id="header",
            className='twelve columns',
        ),
                    
        
        html.Div(
            [
                html.Div(
                [
                    html.Div(
                        [
                            dcc.Dropdown(
                                id='dropdown',
                                options=[{
                                    'label':i,
                                    'value':i
                                    } for i in country],
                                value=df.Country[0],
                                className="dcc_control"
                                ),   
                        ],
                        className="pretty_container"
                    ),
                    
                    html.Div(
                        [
                            html.P("Total no. of Cases"),
                            html.H6(
                                id="cases_text"
                            ) 
                        ],
                        id="cases",
                        className="pretty_container"
                    ),
                    
                     html.Div(
                        [
                            html.P("Total no. of Deaths"),
                            html.H6(
                                id="deaths_text"
                            ) 
                        ],
                        id="deaths",
                        className="pretty_container"
                    ),
                    
                    html.Div(
                        [
                            html.P("Cases - " + today ),
                            html.H6(
                                id="today_text"
                            ) 
                        ],
                        id="today",
                        className="pretty_container"
                    )
                     
                ],
                className="two columns"
                ),
                
                html.Div(
                [
                    dcc.Graph(
                        id='case-by-country',
                    )
                ],
                className="pretty_container nine columns"
                )
            ],
            id='graph',
            
           
        ),
        
        html.Div(
            [
                html.Div(
                    [
                        dcc.Graph(
                            id='map', 
                            figure = fig,
                            )
                        ],
                        className="pretty_container eleven columns"
                    )
                ],
            id="graph-map",
        #    className='row',
            ),
    ],
)
          

#callback to control the graph
@app.callback(Output('case-by-country', 'figure'), [Input('dropdown', 'value')])
def update_graph(country):
    d_fig = df.loc[df['Country'] == country]
    
    fig = px.line(d_fig, x='Date', y='Cases', 
                   title='Number of Cases in ' + str(country))
    
    fig.layout.template = plt_io.templates['presentation']
    fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    
    return fig

@app.callback(Output('cases_text','children'), [Input('dropdown', 'value')])
def update_cases(country):
    d_cases = df.loc[df['Country'] == country]
    total = d_cases['Cases'].sum()
    
    return total

@app.callback(Output('deaths_text','children'), [Input('dropdown', 'value')])
def update_deaths(country):
    d_cases = df.loc[df['Country'] == country]
    total = d_cases['Deaths'].sum()
    
    return total

@app.callback(Output('today_text','children'), [Input('dropdown', 'value')])
def update_today(country):
    d_cases = df.loc[df['Country'] == country]
    d_cases = d_cases.loc[d_cases['Date'] == today]
    total = d_cases['Cases']
    
    return total

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
